In [2]:
import sys
import pandas as pd
import os
sys.path.append("..")

# news scraping
from src.zinfo.news_scraper import NewsScraper

# clustering
from src.zinfo.article_clustering import get_vectorized_titles
from src.zinfo.article_clustering import cluster_articles

# article selecting
from src.zinfo.article_selector import get_best_article_all_clusters

In [2]:
# scraper = NewsScraper('eeeaefaae3c14737bc08e252a6e1991b')
# trending_news = scraper.get_trending_articles_today(5)

In [6]:
trending_news = pd.read_csv("test.csv")
trending_news = trending_news.drop_duplicates(subset=['title'])

In [19]:
os.chdir("..")
os.chdir("data/")

selected_articles = pd.read_csv("selected_articles.csv")
todays_articles = selected_articles.loc[selected_articles.date == "2021-08-12"]

os.chdir("..")
os.chdir("tests/")

In [20]:
todays_articles = selected_articles.loc[selected_articles.date == "2021-08-12"]
print(len(todays_articles))

13


In [22]:
todays_articles

,date,title,url,topic,num_articles
9,2021-08-12,Daywatch: Chicago cop’s death continues to roi...,https://www.chicagotribune.com/news/ct-aud-cb-...,Field of Dreams Game,208
10,2021-08-12,2022 MTV Video Music Award Nominations,https://ohnotheydidnt.livejournal.com/12140694...,Machine Gun Kelly,4
11,2021-08-12,Tom Hanks' son Chet doubles down on his anti-C...,https://www.dailymail.co.uk/news/article-98856...,Tom Hanks son Chet,15
12,2021-08-12,Covid-19 latest updates: FDA set to approve co...,https://www.washingtonpost.com/nation/2021/08/...,Immunocompromised,39
13,2021-08-12,Chloe Bennet leaves live-action Powerpuff Girl...,https://www.gamesradar.com/chloe-bennet-leaves...,Chloe Bennet,4
14,2021-08-12,‘Jeopardy!’ Officially Brings In Mike Richards...,https://hypebeast.com/2021/8/jeopardy-hosts-mi...,Jeopardy new host,17
15,2021-08-12,Furious that LeVar Burton isn't the new 'Jeopa...,https://www.latimes.com/entertainment-arts/tv/...,Jeopardy new host,4
16,2021-08-12,James Wiseman loses mind on Gary Payton II's S...,https://sports.yahoo.com/james-wiseman-loses-m...,Gary Payton II,4
17,2021-08-12,Look to the sky tonight to get a glimpse of th...,https://www.hawaiinewsnow.com/2021/08/12/look-...,Meteor shower tonight,11
18,2021-08-12,Who Is 'Kissing Booth' Star Joey King Dating a...,https://www.newsweek.com/joey-king-dating-boyf...,Kissing Booth 3,3
